In [1]:
import pandas as pd
import numpy as np

In [3]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [5]:
df = pd.read_csv('covid_toy.csv');

In [7]:
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [9]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [11]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df.drop(columns=['has_covid']),
                                                   df['has_covid'],
                                                   test_size=0.2)

In [13]:
x_train

,age,gender,fever,cough,city
7,20,Female,NaN,Strong,Mumbai
48,66,Male,99.0,Strong,Bangalore
17,40,Female,98.0,Strong,Delhi
35,82,Female,102.0,Strong,Bangalore
52,47,Female,100.0,Strong,Bangalore
...,...,...,...,...,...
57,49,Female,99.0,Strong,Bangalore
41,82,Male,NaN,Mild,Kolkata
92,82,Female,102.0,Strong,Kolkata
82,24,Male,98.0,Mild,Kolkata


In [31]:
# adding simple imputer to fever col to fill missing cols
si = SimpleImputer()
x_train_fever = si.fit_transform(x_train[['fever']])
x_test_fever = si.fit_transform(x_test[['fever']])
x_train_fever.shape

(80, 1)

In [33]:
# Ordinary Encoding -> cough
oe = OrdinalEncoder(categories=[['Mild','Strong']])
x_train_cough = oe.fit_transform(x_train[['cough']])
x_test_cough = oe.fit_transform(x_test[['cough']])
x_train_cough.shape

(80, 1)

In [35]:
# OneHotEncoding -> gender, city
ohe = OneHotEncoder(drop='first', sparse_output=False)
x_train_gender_city = ohe.fit_transform(x_train[['gender','city']])
x_test_gender_city = ohe.fit_transform(x_test[['gender','city']])
x_train_gender_city.shape

(80, 4)

In [37]:
#Extracting age
x_train_age = x_train.drop(columns=['gender','fever','cough','city']).values
x_test_age = x_test.drop(columns=['gender','fever','cough','city']).values

x_train_age.shape

(80, 1)

In [39]:
x_train_transformed = np.concatenate((x_train_age,x_train_fever,x_train_gender_city, x_train_cough), axis=1)
x_test_transformed = np.concatenate((x_test_age,x_test_fever,x_test_gender_city, x_test_cough), axis=1)

x_train_transformed.shape

(80, 7)

# using Column Transformer

In [49]:
from sklearn.compose import ColumnTransformer

In [51]:
transformer = ColumnTransformer(transformers=[
    ('tnf1', SimpleImputer(),['fever']),
    ('tnf2', OrdinalEncoder(categories=[['Mild','Strong']]), ['cough']),
    ('tnf3', OneHotEncoder(sparse_output=False, drop='first'), ['gender','city'])
], remainder='passthrough')

In [53]:
transformer.fit_transform(x_train).shape

(80, 7)